# MIRCF Results Reproduction — Section 6

This notebook reproduces all empirical results reported in **Section 6** of the
*Meta-Intentional Reflexive Cybernetic Framework (MIRCF)* paper.

Each Research Question (**RQ1–RQ6**) is evaluated in a **dedicated, deterministic section**.
All figures and tables are generated programmatically and exported in **SVG** and **300-dpi JPG** formats suitable for journal submission.

## Design Principles
- Deterministic execution (fixed random seeds)
- One Research Question per section
- Explicit metric → figure → claim traceability
- No hidden state or UI dependencies


## Optional Dependency Installation
Run this cell **only** in fresh environments.


In [ ]:
%pip install --quiet numpy pandas scipy matplotlib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import sem
import warnings, os

warnings.filterwarnings("ignore")
plt.rcParams.update({"figure.dpi": 300, "font.size": 11})

EXPORT_DIR = "outputs"
FIG_DPI = 300
os.makedirs(EXPORT_DIR, exist_ok=True)

In [ ]:
def abel_regularize(x, eps):
    return x / (1 + eps * np.abs(x))

def EDB_scalar(x, sigma=0.1):
    p = 0.5 * (1 + np.tanh(x / sigma))
    p = np.clip(p, 1e-9, 1 - p)
    return -p * np.log2(p) - (1 - p) * np.log2(1 - p)

def spectral_gap(A):
    vals = np.sort(np.real_if_close(np.linalg.eigvals(A)))[::-1]
    return vals[0] - vals[1] if len(vals) > 1 else 0.0

def row_normalize(A):
    rs = A.sum(axis=1, keepdims=True)
    rs[rs == 0] = 1.0
    return A / rs

In [ ]:
def generate_adj(N, mode, p=0.1, d=4, seed=0):
    rng = np.random.default_rng(seed)
    A = np.zeros((N, N))
    if mode == "ring":
        for i in range(N):
            A[i, (i+1)%N] = A[i, (i-1)%N] = 1
    elif mode == "erdos_renyi":
        A = (rng.random((N, N)) < p).astype(float)
        A = np.triu(A, 1); A += A.T
    elif mode == "fully_connected":
        A[:] = 1; np.fill_diagonal(A, 0)
    elif mode == "ramanujan_bipartite":
        N1 = N // 2
        for i in range(N1):
            nbrs = rng.choice(np.arange(N1, N), size=d, replace=False)
            for j in nbrs:
                A[i, j] = A[j, i] = 1
    return A

In [ ]:
def mircf_step(x_t, x_tm1, a, b, eps, K_RSI, K_EDB, gamma, W):
    x_raw = a * x_t + b * x_tm1
    x_reg = abel_regularize(x_raw, eps)
    rsi = np.exp(-0.1 * np.abs(x_reg - x_t))
    edb = np.array([EDB_scalar(v) for v in x_reg])
    u_rsi = -K_RSI * np.maximum(0, 0.5 - rsi) * x_reg
    u_edb = -K_EDB * edb * np.sign(x_reg)
    coupling = gamma * (W @ x_t - x_t)
    return x_reg + u_rsi + u_edb + coupling, rsi, edb

In [ ]:
def run_mircf(cfg, seed):
    rng = np.random.default_rng(seed)
    N, T = cfg['N'], cfg['T']
    A = generate_adj(N, cfg['mode'], cfg.get('p', 0), cfg.get('d', 4), seed)
    W = row_normalize(A)
    X = np.zeros((T, N)); RSI = np.zeros((T, N)); EDB = np.zeros((T, N)); V = np.zeros(T)
    X[0] = rng.normal(0, 0.6, N); X[1] = rng.normal(0, 0.6, N)
    for t in range(1, T-1):
        X[t+1], RSI[t+1], EDB[t+1] = mircf_step(
            X[t], X[t-1], cfg['a'], cfg['b'], cfg['eps'],
            cfg['K_RSI'], cfg['K_EDB'], cfg['gamma'], W)
        mu = X[t+1].mean()
        V[t+1] = 0.5*np.sum((X[t+1]-mu)**2) + np.sum(np.maximum(0,0.5-RSI[t+1])**2)
    return dict(X=X, RSI=RSI, EDB=EDB, V=V, A=A)

## RQ1 — Global Stability

In [ ]:
cfg = dict(N=24, T=300, mode='erdos_renyi', p=0.12,
           a=1.02, b=0.98, eps=0.02,
           gamma=0.12, K_RSI=0.45, K_EDB=0.35)
sim = run_mircf(cfg, seed=3)
plt.plot(sim['V']); plt.title('RQ1 — Global Stability')
plt.savefig(f'{EXPORT_DIR}/RQ1_stability.svg')
plt.savefig(f'{EXPORT_DIR}/RQ1_stability.jpg', dpi=FIG_DPI); plt.show()

## RQ2 — Entropy Regulation

In [ ]:
plt.plot(sim['EDB'].mean(axis=1)); plt.title('RQ2 — Entropy Bound')
plt.savefig(f'{EXPORT_DIR}/RQ2_entropy_bound.svg')
plt.savefig(f'{EXPORT_DIR}/RQ2_entropy_bound.jpg', dpi=FIG_DPI); plt.show()

## RQ3 — Topological Resonance

In [ ]:
topos=['ring','erdos_renyi','ramanujan_bipartite']
gaps=[spectral_gap(run_mircf(dict(**cfg,mode=t),3)['A']) for t in topos]
plt.bar(topos,gaps); plt.title('RQ3 — Resonance Metric')
plt.savefig(f'{EXPORT_DIR}/RQ3_topology_rm.svg')
plt.savefig(f'{EXPORT_DIR}/RQ3_topology_rm.jpg', dpi=FIG_DPI); plt.show()

## RQ4 — Ablation Analysis

In [ ]:
for k in [(0.45,0.35),(0,0.35),(0.45,0)]:
    cfg['K_RSI'],cfg['K_EDB']=k
    plt.plot(run_mircf(cfg,3)['V'])
plt.title('RQ4 — Ablation'); plt.show()

## RQ5-B — Quantum Reflexive Correspondence

In [ ]:
def bloch_to_rho(b):
    x,y,z=b; I=np.eye(2)
    sx=np.array([[0,1],[1,0]]); sy=np.array([[0,-1j],[1j,0]]); sz=np.array([[1,0],[0,-1]])
    rho=0.5*(I+x*sx+y*sy+z*sz)
    return rho/np.trace(rho)
def purity(rho): return np.real(np.trace(rho@rho))
cfg['N']=24; simq=run_mircf(cfg,3)
X,RSI,EDB=simq['X'],simq['RSI'],simq['EDB']
rho=bloch_to_rho([0.2,0.05,np.tanh(X[1,0])])
p=[purity(rho:=rho) for _ in range(len(X))]
plt.plot(RSI[:,0],label='RSI'); plt.plot(p,label='Purity'); plt.legend(); plt.show()

## RQ6 — Counterfactual Failure

In [ ]:
fail=dict(N=48,T=200,mode='fully_connected',a=1.3,b=0.98,eps=1e-6,gamma=0.25,K_RSI=0,K_EDB=0)
plt.plot(run_mircf(fail,1)['V']); plt.title('RQ6 — Unstable Fail'); plt.show()